In [6]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone import ViewField as F

In [8]:
# name = "dental_rtg_merged_full"
name = 'foo4'
data_path = "/home/kuntik/dev/datasets/dental_merged/images"
# labels_path = "/home/kuntik/dev/datasets/dental_merged/annotations.json"
labels_path = "/home/kuntik/dev/datasets/dental_rtg_merged2/annotations.json"
dataset = fo.Dataset.from_dir(
    dataset_type = fo.types.COCODetectionDataset,
    data_path = data_path,
    labels_path = labels_path,
    name=name
)
# dataset = fo.load_dataset('dental_rtg_merged_f')
# dataset = fo.load_dataset('dental_caries_dataset_merged2')


 100% |███████████████| 2599/2599 [3.5s elapsed, 0s remaining, 777.0 samples/s]      


In [2]:
dataset = fo.load_dataset('foo3')

In [25]:
print(dataset)
session = fo.launch_app(dataset,desktop=True)

Name:        foo4
Media type:  image
Num samples: 2599
Persistent:  False
Tags:        ['train', 'val']
Sample fields:
    id:              fiftyone.core.fields.ObjectIdField
    filepath:        fiftyone.core.fields.StringField
    tags:            fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    new_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Desktop App launched.


In [6]:
fouc.add_coco_labels(
    dataset,
    "old_gt",
    "/home/kuntik/dev/datasets/dental_merged/annotations.json",
)

In [10]:
session.view=dataset.view()

In [18]:
# dataset.take()
# dataset.filter_labels('predictions', F("confidence") > 0.15).evaluate_detections(
results = dataset.evaluate_detections(
    "new_predictions",
    gt_field="ground_truth",
    iou=0.5,
    compute_mAP = True,
    # eval_key="eval"
)

Evaluating detections...
 100% |███████████████| 2599/2599 [35.5s elapsed, 0s remaining, 76.0 samples/s]      
Performing IoU sweep...
 100% |███████████████| 2599/2599 [51.0s elapsed, 0s remaining, 48.5 samples/s]      


In [19]:
results.mAP()

0.3517202559597863

In [10]:

session.view = dataset.sort_by('eval_fn', reverse=True)
# session.view = dataset.view()


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/fiftyone/core/client.py", line 125, in run_client
    io_loop.run_sync(connect)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/tornado/ioloop.py", line 530, in run_sync
    return future_cell[0].result()
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/fiftyone/core/client.py", line 92, in connect
    message = json_util.loads(message)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/bson/json_util.py", line 439, in loads
    return json.loads(s, *args, **kwargs)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3


Session disconnected, trying to reconnect


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Cou

In [28]:
pred_name = 'new_predictions'
val_v = dataset.match_tags('val')
val_v15 = val_v.filter_labels(pred_name, F('confidence')>0.30)
results = val_v15.evaluate_detections(
    pred_name,
    gt_field="ground_truth",
    iou=0.5,
    compute_mAP = True
)
results.print_report()
print(results.mAP())


Evaluating detections...
 100% |█████████████████| 304/304 [1.1s elapsed, 0s remaining, 302.1 samples/s]         
Performing IoU sweep...
 100% |█████████████████| 304/304 [1.3s elapsed, 0s remaining, 235.4 samples/s]         
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
       decay       0.71      0.61      0.65       809

   micro avg       0.71      0.61      0.65       809
   macro avg       0.35      0.30      0.33       809
weighted avg       0.71      0.61      0.65       809

0.21627208903642428


In [18]:
results.
results.print_report()
results.mAP()


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
       decay       0.06      0.91      0.11      4477

   micro avg       0.06      0.91      0.11      4477
   macro avg       0.03      0.45      0.05      4477
weighted avg       0.06      0.91      0.11      4477



0.23425058387481193

In [11]:
dataset_15 = dataset.filter_labels('new_predictions', F("confidence") > 0.15)
dataset_15 = dataset.evaluate_detections(
    "new_predictions",
    gt_field = "ground_truth",
    iou=0.5,
    compute_mAP = True,
    eval_key='eval_new'
)

NameError: name 'val_v' is not defined

In [31]:
results_val_20.print_report()

results_val_10.ious

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
       decay       0.47      0.50      0.49       766
 restoration       0.00      0.00      0.00         0

   micro avg       0.47      0.50      0.49       766
   macro avg       0.16      0.17      0.16       766
weighted avg       0.47      0.50      0.49       766



array([0.6571302783345557, 0.7207949249093901, 0.5790841866967978, ...,
       0.5252533933306285, None, 0.6266584390100287], dtype=object)

In [30]:
path = "/home/kuntik/dev/datasets/dental_merged/images/"
dataset[[path+'100.png', path + '101.png']]

InvalidId: '/home/kuntik/dev/datasets/dental_merged/images/100.png' is not a valid ObjectId, it must be a 12-byte input or a 24-character hex string

In [48]:
import fiftyone.brain as fob
fob.compute_mistakenness(
    supressed_view15,
    'predictions',
    label_field='ground_truth',
    use_logits=False
)

Evaluating detections...
 100% |█████████████████| 944/944 [10.8s elapsed, 0s remaining, 79.2 samples/s]      
Computing mistakenness...
 100% |█████████████████| 944/944 [5.2s elapsed, 0s remaining, 182.4 samples/s]      
Mistakenness computation complete


In [42]:
anno_key = "n3"

label_schema = {
    "ground_truth" : {
        "type" : "detections",
        "classes" : ["decay"]
    },
    "predictions" : {
        "type" : "detections",
        "classes" : ["decay"]
    }
}

# Upload the samples and launch CVAT
annot_res = dataset[1800:].annotate(
    anno_key=anno_key,
    label_schema=label_schema,
    attributes=False,
    image_quality=100,
    segment_size=100
)

# anno_results = unique_5_view.annotate(
#     anno_key,
#     label_field='ground_truth',
#     label_type='detections',
#     attributes=False,
#     classes = ['decay'],
#     launch_editor= False,
#     # classes = ['decay']
# )


Uploading samples to CVAT...
Upload complete


In [34]:

# match = (F{"p"})
# d15 = dataset.filter_labels('predictions', F("confidence") > 0.15)
# d15 = dataset.filter_labels('predictions', F("confidence") > 0.15 )
d15 = dataset.exclude_labels('predictions', F('confidence') < 0.15)


# d15 = dataset.filter_detections('predictions', F("confidence") > 0.15)

TypeError: string indices must be integers

In [32]:
t= dataset.take(1)


# print(d15)
# d15.

In [25]:
print(d15)

Dataset:     dental_rtg_merged
Media type:  image
Num samples: 1863
Tags:        ['train', 'valid']
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:      fiftyone.core.fields.IntField
    eval_fp:      fiftyone.core.fields.IntField
    eval_fn:      fiftyone.core.fields.IntField
View stages:
    1. FilterDetections(field='predictions', filter={'$gt': ['$$this.confidence', 0.15]}, only_matches=True)


In [24]:
# dataset[0]
import json
from pathlib import Path
with open('rtg2_preds4.json', 'r') as json_file:
    preds = json.load(json_file)
# dataset['/home/kuntik/carries_dataset/images/1.png']
data_path = '/home/kuntik/dev/datasets/dental_merged/images'
add_prediction(preds, dataset, Path(data_path))

In [23]:
from PIL import Image
def add_prediction(pred_dict, dataset, img_root):
    # for sample in dataset:
    det_id = 0
    for pred_key, pred_value in pred_dict.items():
        dets = []
        # id = sample.id
        path = img_root / pred_key
        sample = dataset[str(path)]
        s_id = sample.id
        scores = pred_value['scores']
        labels = pred_value['labels']
        bboxes = pred_value['bboxes']
        img = Image.open(path)
        for box, score, label in zip(bboxes, scores, labels):
            #TODO to limit confidence
            # if score < 0.15:
            #     continue
            coco_obj = fouc.COCOObject(det_id, s_id, int(label), [box[0],box[1], box[2]-box[0], box[3]-box[1]], score=score)
            det = coco_obj.to_detection(img.size, ['decay'])
            dets.append(det)
            det_id += 1
        detection = fo.Detections(detections=dets)
        sample['new_predictions'] = detection
        # sample.tags.append(pred_value['stage'])
        sample.tags = [pred_value['stage']]
        sample.save()
        



In [22]:
from PIL import Image
img = Image.open('/home/kuntik/carries_dataset/images/1.png')
img.size

(1068, 847)